In [28]:
import pandas as pd, numpy as np
from torch.utils.data import Dataset
import torch
LOCAL = True
if LOCAL:
    dataPath = '/home/zhenlan/Desktop/Projects/HMS/Data/'

In [2]:
df = pd.read_csv(dataPath+'train.csv')
TARGETS = df.columns[-6:].tolist()

In [5]:
# normalize prob
df[TARGETS] = df[TARGETS].values/df[TARGETS].sum(1).values[:,None]

In [11]:
out = df.groupby('eeg_id').agg({'eeg_label_offset_seconds': list,
                                    'seizure_vote': list,
                                    'lpd_vote': list,
                                    'gpd_vote': list,
                                    'lrda_vote': list,
                                    'grda_vote': list,
                                    'other_vote': list
                                }).reset_index()

In [29]:
out

,eeg_id,eeg_label_offset_seconds,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote
0,568657,"[0.0, 6.0, 12.0, 16.0]","[0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0]","[0.25, 0.25, 0.25, 0.25]","[0.0, 0.0, 0.0, 0.0]","[0.16666666666666666, 0.16666666666666666, 0.1...","[0.5833333333333334, 0.5833333333333334, 0.583..."
1,582999,"[0.0, 4.0, 10.0, 14.0, 16.0, 18.0, 20.0, 22.0,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.8571428571428571, 0.8571428571428571, 0.857...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.07142857142857142, 0.07142857142857142, 0.0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.07142857142857142, 0.07142857142857142, 0.0..."
2,642382,"[0.0, 24.0]","[0.0, 0.0]","[0.0, 0.0]","[0.0, 0.0]","[0.0, 0.0]","[0.0, 0.0]","[1.0, 1.0]"
3,751790,[0.0],[0.0],[0.0],[1.0],[0.0],[0.0],[0.0]
4,778705,[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[1.0]
...,...,...,...,...,...,...,...,...
17084,4293354003,[0.0],[0.0],[0.0],[0.0],[0.0],[0.5],[0.5]
17085,4293843368,[0.0],[0.0],[0.0],[0.0],[0.0],[0.5],[0.5]
17086,4294455489,[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[1.0]
17087,4294858825,"[0.0, 2.0, 6.0, 8.0, 12.0]","[0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0]","[0.06666666666666667, 0.06666666666666667, 0.0...","[0.9333333333333333, 0.9333333333333333, 0.933..."


In [59]:
class eegData(Dataset):
    def __init__(self, df):
        self.df = df
        self._load_eegs()
        
    def _load_eegs(self):
        self.eegs = [pd.read_parquet(dataPath+'train_eegs/'+str(id)+'.parquet').values\
                      for id in self.df.eeg_id.tolist()]
        
    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self,idx):
        out = self.df.iloc[idx,1:].tolist()
        offset,target = self.sample_time(out)
        eeg = self.eegs[idx][offset*200:(offset+50)*200]
        return torch.tensor(eeg,dtype=torch.float32),target
    
    @staticmethod
    def sample_time(out):
        time = np.random.randint(0,len(out[0]))
        offset, *target = [o[time] for o in out]
        return int(offset),torch.tensor(target,dtype=torch.float32)

In [60]:
eegdata = eegData(out)

In [68]:
eeg,target = eegdata[150]